In [14]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [15]:
data_dir = 'Maskset'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.CenterCrop(224),
                                     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])
test_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.CenterCrop(224),
                                     transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
valid_data = datasets.ImageFolder(data_dir + '/Validation', transform=test_transforms)


trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32)

In [16]:
model = models.vgg16(pretrained = True)

In [17]:
device = torch.device('cuda')
for params in model.parameters() :
    params.requires_grad = False
    
from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088,500)),
    ('relu1', nn.ReLU()),
    ('dropout', nn.Dropout(p = 0.2)),
    ('fc2', nn.Linear(500,102)),
    ('output', nn.LogSoftmax(dim=1))    
    
]))

model.classifier = classifier

In [18]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr = 0.001)

In [19]:
model.cuda();
def validation(model,criterion, testloader):
    testloss = 0
    accuracy = 0
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)

        output = model.forward(images)
        testloss += criterion(output,labels).item()

        ps = torch.exp(output)
        equality = (labels.data ==ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    return testloss, accuracy

In [21]:

import time
epochs = 2
print_every = 10
steps = 0
running_loss = 0

stp=[]
ts_loss = []
tr_loss = []
accu = []
#total_time = time.time()
for e in range(epochs):
    model.train()
    
    for images,labels in trainloader:
        steps += 1
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logits = model.forward(images)
        loss = criterion(logits, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            with torch.no_grad():
                testloss, accuracy = validation(model,criterion, testloader)
                
                
            print('Epochs {}/{} '.format(e+1, epochs),
                  'train loss {:.3f} '.format(running_loss/print_every),
                  'test loss {:.3f} '.format(testloss/len(testloader)),
                  'accuracy {:.3f} '.format(accuracy/len(testloader)))
            
            tr_loss.append(running_loss/print_every)     
            stp.append(steps)    
            ts_loss.append(testloss/len(testloader))
            accu.append(accuracy/len(testloader))
            
            running_loss = 0
            model.train()

#print("total_time {} minutes".format(time.time()-total_time())/60)            

RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 4.00 GiB total capacity; 2.85 GiB already allocated; 0 bytes free; 2.87 GiB reserved in total by PyTorch)

In [ ]:
plt.plot(stp, tr_loss, label = 'training')
plt.plot(stp,ts_loss, label = 'testing')
plt.plot(stp, accu, label = 'accuracy')
plt.legend()

In [ ]:
model.class_to_idx = train_data.class_to_idx
check = {
        'arch':'vgg16',
         'optimizer': optimizer.state_dict,
        'state_dict': model.state_dict(),
        'class_to_idx':model.class_to_idx
}
torch.save(check, 'checkpoint5.pth')

In [ ]:
model = models.vgg16(pretrained = True)
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088,500)),
    ('relu1', nn.ReLU()),
    ('dropout', nn.Dropout(p = 0.2)),
    ('fc2', nn.Linear(500,102)),
    ('output', nn.LogSoftmax(dim=1))]))
model.classifier = classifier
model.class_to_idx = check['class_to_idx']
model.load_state_dict(check['state_dict'])

In [ ]:
device = torch.device('cuda')
model.cuda();
def check_accuracy():
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in validloader:
            images, labels = images.to(device), labels.to(device)
            output = model.forward(images)
            _, predicted = torch.max(output.data,1)
            total +=labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print("accuracy : {}%".format(100*correct//total))
    

In [ ]:
check_accuracy()

In [ ]:
from PIL import Image
img = Image.open('Maskset/Train/1/0006.jpg')
img

In [ ]:
import json

with open('mask_name.json', 'r') as f:
    mask_name = json.load(f)

In [ ]:
import numpy as np
def process_image(path):
    image = Image.open(path)
    
    transformer = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
    np_image = transformer(image).float()
    return np_image

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
imshow(process_image('Maskset/Train/1/0006.jpg'))

In [ ]:
model.cuda()
device = torch.device('cuda')
def predict(path,model):
    img = process_image(path)
    img = img.float().unsqueeze_(0)
    img = img.to(device)
    with torch.no_grad():
        output = model.forward(img)
    ps = torch.exp(output)
    probs, indices = ps.topk(2)
    probs = probs.cpu().numpy()[0]
    indices = indices.cpu().numpy()[0]
    class_to_idx = {v:k for k,v in model.class_to_idx.items()}
    classes = [class_to_idx[x] for x in indices]
    return probs, classes

In [ ]:
predict('Maskset/Train/1/0006.jpg',model)

In [ ]:
path = 'Maskset/Train/1/0006.jpg'
probs, classes = predict(path,model)

mask_names = [mask_name[str(i)] for i in classes]

primg = process_image(path)
#sub1
plt.figure(figsize = (10,10))
ax1 = plt.subplot(222)
img = Image.open(path)
ax1.set_title('Image of the test flower')
ax1.imshow(img)

#sub2 
ax2 = plt.subplot(221)
yticks = np.arange(2)
ax2.set_yticks(yticks)
ax2.set_yticklabels(mask_names)
ax2.set_title('probability chart')
ax2.set_xlabel('probability')
ax2.set_ylabel('flower type')

ax2.barh(yticks, probs, align = 'center')

image = process_image(path)
imshow(primg)